In [ ]:
pip install selenium

In [10]:
handle = '' #인스타그램 아이디
pwd = ''  #인스타그램 비밀번호
keyword= '여행'   #검색할 키워드
num_of_pages = 10 #몇 번 스크롤 할 건지; e.g. 200 --> 1730개 게시글 긁어짐

In [11]:
#함수 정의
from selenium import webdriver
from urllib.parse import quote
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd 
import re

#URL 가져오기
def InstagramUrlFromKeyword (browser,keyword,num_of_pagedowns):
    keyword_url_encode=quote(keyword)
    url='https://www.instagram.com/explore/tags/'+keyword+'/?hl=ko'
    browser.get(url)
    time.sleep(3)
    login = browser.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button')
    login.click()
    time.sleep(3)
    user_id = browser.find_element_by_class_name('_2hvTZ.pexuQ.zyHYP')
    pw = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[3]/div/label/input')
    user_id.send_keys(handle)
    pw.send_keys(pwd)
    button = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button')
    button.click()
    time.sleep(3)
    arr_href=[]
    body=browser.find_element_by_tag_name('body')
    for i in range(num_of_pagedowns):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        post=browser.find_elements_by_class_name('v1Nh3')
        for j in post:
            href_str=j.find_element_by_css_selector('a').get_attribute('href')
            arr_href.append(href_str)
    return set(arr_href)    

char = r'#[\d\w]+'
char2 = re.compile('[^ 0-9a-zA-Zㄱ-ㅣ가-힣!#?]')

#URL에서 데이터 가져오기
def IdHashTagFromInstagram(browser,url):
    browser.get(url)
    datetime = browser.find_element_by_class_name('_1o9PC.Nzb55').get_attribute('title')
    href=browser.find_elements_by_class_name('C4VMK')
    #포스트 내용(글, 태그)
    total_hash_text=[]    
    for i in range(0, len(href)):
        hash_text= href[i].find_element_by_css_selector('span').text
        total_hash_text.append(hash_text)
    tags = re.findall(char, char2.sub(' ',str(total_hash_text)))
    user_name = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[1]/div[1]/h2/a').text
    like_count = int(browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/div[2]/section[2]/div/div/button/span').text)
    
    return datetime, user_name, like_count, tags

In [12]:
#크롤링
browser= webdriver.Chrome()
arr=InstagramUrlFromKeyword(browser,keyword,num_of_pages)
insta_df=pd.DataFrame(columns = {'Datetime', 'User_Name', 'Like_Count', 'Tags'})

for url in arr:
        try:
            datetime, user_name, like_count, tags = IdHashTagFromInstagram(browser,url)
            insta_df = insta_df.append({'Datetime': datetime, 'User_Name':user_name, 'Like_Count':like_count,'Tags': tags}, ignore_index = True)
        except:
            continue

In [17]:
insta_df

,Like_Count,Tags,Datetime,User_Name
0,39,[],2019년 12월 27일,hyang_x2
1,14,"[#성수, #스피닝울프성수, #재즈바, #명당자리, #2019마무리, #고생했엉, ...",2019년 12월 27일,elizabeth_wyh
2,113,[],2019년 12월 26일,jingmihou
3,20,"[#서울, #서울여행, #사울카페, #성수, #성수동카페, #성수카페, #성수아모레...",2019년 12월 27일,yese50121
4,7,"[#성수, #윤경양식당, #맛있오, #양이짝아]",2019년 12월 27일,kim_yk7
5,31,"[#고생해쪄성으니, #연구원, #가운, #10년차, #직장인, #요가강사, #yog...",2019년 12월 27일,happy_ssungeun_itihasa
6,8,"[#appleiphonexs, #어린이대공원, #어린이대공원출사, #성수, #성수동...",2019년 12월 27일,sosouniquee_
7,9,"[#카페봇, #서울카페, #커피, #커피스타그램, #카페, #카페스타그램, #서울,...",2019년 12월 27일,annygraph
8,5,"[#달고나, #차a, #성수, #달고나밀크, #달고나오롱차밀크티, #달고나스콘, #...",2019년 12월 27일,alegre_eve.yon
9,530,[],2019년 12월 19일,didi_u3


In [ ]:
#csv 저장
insta_df.to_csv(keyword + '.csv',mode='w', encoding='euc-kr')

In [19]:
#빈도분석
from collections import Counter

insta_str = []
for tags in insta_df['Tags']:
    insta_str.extend(tags)
    
c = Counter(insta_str)

c.most_common(50)